In [2]:
# create a machine learning algorithm to see if a customer will buy again from an audiobook app

In [7]:
import numpy as np
from sklearn import preprocessing
import pandas as pd

data = pd.read_csv(r'C:\Users\Dylan\Desktop\ML\AudioBookData.csv')

In [12]:
# load w pandas to see if need to make changes

# looks decent, needs standardization 

data

,873,2160,2160.1,10.13,10.13.1,0,8.91,0.1,0.2,0.3,0.4,1
0,611,1404.0,2808,6.66,13.33,1,6.50,0.00,0.0,0,182,1
1,705,324.0,324,10.13,10.13,1,9.00,0.00,0.0,1,334,1
2,391,1620.0,1620,15.31,15.31,0,9.00,0.00,0.0,0,183,1
3,819,432.0,1296,7.11,21.33,1,9.00,0.00,0.0,0,0,1
4,138,2160.0,2160,10.13,10.13,1,9.00,0.00,0.0,0,5,1
...,...,...,...,...,...,...,...,...,...,...,...,...
14078,27398,2160.0,2160,7.99,7.99,0,8.91,0.00,0.0,0,54,0
14079,28220,1620.0,1620,5.33,5.33,1,9.00,0.61,0.0,0,4,0
14080,28671,1080.0,1080,6.55,6.55,1,6.00,0.29,0.0,0,29,0
14081,31134,2160.0,2160,6.14,6.14,0,8.91,0.00,0.0,0,0,0


In [13]:
# column meanings

# 0.) Index (pandas created)
# 1.) Customer ID
# 2.) Book Length overall
# 3.) Book length used
# 4.) Price
# 5.) Price average
# 6.) Review (0 1 binary yes/no)
# 7.) Rating of that review 0 if not
# 8.) Minutes Listened
# 9.) Completion %
# 10.) Support Requests
# 11.) Last visited minute purchase date (days inactivte)
# 12.) Binary Y/N if bought in last 6 months

In [17]:
convertdata = np.loadtxt(r'C:\Users\Dylan\Desktop\ML\AudioBookData.csv', delimiter = ',')
converted = convertdata[:,1:-1]
targetsALL = convertdata[:,-1]

# Balance The Data

In [27]:
# total list
numofonetargets = int(np.sum(targetsALL))


# create the counter
counter = 0

# storage
indices_to_remove = []

# loop through all
for i in range (targetsALL.shape[0]):
    
    # find the values equal to 0
    if targetsALL[i] == 0:
    
    # increase the counter   
        
        counter +=1
        
        if counter > numofonetargets:
            indices_to_remove.append(i)

            
unscaled_inputs_equal_priors = np.delete(converted, indices_to_remove, axis = 0)
targets_equal_priors = np.delete(targetsALL, indices_to_remove, axis = 0)

# Standardize

In [28]:
scaled_inputs = preprocessing.scale(unscaled_inputs_equal_priors)

# Shuffle

In [29]:
shuffled_indices = np.arange(scaled_inputs.shape[0])
np.random.shuffle(shuffled_indices)


shuffled_inputs = scaled_inputs[shuffled_indices]
shuffled_targets = targets_equal_priors[shuffled_indices]

# Split Data

In [33]:
samples_count = shuffled_inputs.shape[0]

# 80 10 10 split

# counts

train_samples_count = int(.8 * samples_count)
vali_samples_count = int(.1 * samples_count)
test_samples_count = int(.1 * samples_count)

# extract from main


train_inputs = shuffled_inputs[:train_samples_count]
train_targets = shuffled_targets[:train_samples_count]

vali_inputs = shuffled_inputs[train_samples_count:train_samples_count+vali_samples_count]
vali_targets = shuffled_targets[train_samples_count:train_samples_count+vali_samples_count]


test_inputs = shuffled_inputs[train_samples_count+vali_samples_count:]
test_targets = shuffled_targets[train_samples_count+vali_samples_count:]

print(train_samples_count)
print(vali_samples_count)
print(test_samples_count)

3579
447
447


In [34]:
np.savez('AudioBook_train', inputs=train_inputs, targets=train_targets)
np.savez('AudioBook_vali', inputs=vali_inputs, targets=vali_targets)
np.savez('AudioBook_test', inputs=test_inputs, targets=test_targets)

In [35]:
import tensorflow as tf

C:\Users\Dylan\Anaconda3\envs\py3-TF2.0\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\Dylan\Anaconda3\envs\py3-TF2.0\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\Dylan\Anaconda3\envs\py3-TF2.0\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\Dylan\Anaconda3\envs\py3-TF2.0\lib\site-packages\ten

# loads (incase want different files, model is highly re-usable)

In [37]:
npz = np.load('AudioBook_train.npz')

train_inputs = npz['inputs'].astype(np.float)
train_targets = npz['targets'].astype(np.int)

npztemp = np.load('AudioBook_vali.npz')

validation_inputs = npztemp['inputs'].astype(np.float)
validation_targets = npztemp['targets'].astype(np.int)

npztemp1 = np.load('AudioBook_test.npz')

test_inputs = npztemp1['inputs'].astype(np.float)
test_targets = npztemp1['targets'].astype(np.int)

# Model

In [42]:
#Hyperparameters
input_size = 10
output_size = 2
hidden_layer_size = 50

# Sequential used to start layering the model (STACK LAYERS)

model = tf.keras.Sequential([
    
    # dot product math to find the bias and weights + activation function
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
    # second layer
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
    # output layer want probabilities so use softmax
    tf.keras.layers.Dense(output_size, activation='softmax'),
                            ])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

EPOCHS = 100
BATCH_SIZE = 100


early_stopping = tf.keras.callbacks.EarlyStopping(patience=2)

model.fit(train_inputs,
         train_targets,
         batch_size = BATCH_SIZE,
         epochs = EPOCHS,
        callbacks=[early_stopping],
         validation_data = (validation_inputs, validation_targets),
         verbose = 2)

Train on 3579 samples, validate on 447 samples
Epoch 1/100
3579/3579 - 0s - loss: 0.4900 - accuracy: 0.8335 - val_loss: 0.3876 - val_accuracy: 0.8613
Epoch 2/100
3579/3579 - 0s - loss: 0.3377 - accuracy: 0.8810 - val_loss: 0.3205 - val_accuracy: 0.8725
Epoch 3/100
3579/3579 - 0s - loss: 0.2964 - accuracy: 0.8927 - val_loss: 0.3041 - val_accuracy: 0.8814
Epoch 4/100
3579/3579 - 0s - loss: 0.2779 - accuracy: 0.9003 - val_loss: 0.2979 - val_accuracy: 0.8770
Epoch 5/100
3579/3579 - 0s - loss: 0.2690 - accuracy: 0.8994 - val_loss: 0.2972 - val_accuracy: 0.8747
Epoch 6/100
3579/3579 - 0s - loss: 0.2605 - accuracy: 0.9030 - val_loss: 0.2864 - val_accuracy: 0.8792
Epoch 7/100
3579/3579 - 0s - loss: 0.2555 - accuracy: 0.9042 - val_loss: 0.2828 - val_accuracy: 0.8837
Epoch 8/100
3579/3579 - 0s - loss: 0.2499 - accuracy: 0.9053 - val_loss: 0.2788 - val_accuracy: 0.8814
Epoch 9/100
3579/3579 - 0s - loss: 0.2478 - accuracy: 0.9064 - val_loss: 0.2749 - val_accuracy: 0.8814
Epoch 10/100
3579/3579 - 0

In [43]:
# we can classify 89.26 % of customers on if they will return or not

# Test

In [44]:
test_loss,  test_accuracy = model.evaluate(test_inputs, test_targets)

448/448 [==============================] - 0s 54us/sample - loss: 0.2425 - accuracy: 0.9129


applying this new model to the business ..

model.predict_on_batch(*company gives data on a potential canidate*)

model returns 0 or 1, telling marketing team to invest on advertising or not.